## 024-Genome Assembly as Shortest Superstring

### ゲノムシーケンス入門

同一ゲノムの複数コピーを小さな断片（リード）に分割し、それらを計算機的に再構築することでゲノム配列を復元する。

「GC含量の計算」で見たように、人間のゲノム配列の約99.9%はほぼ共通している。したがって、もしある種の完全長ゲノムがいくつか解明されれば、その種のゲノム全体を解読する重要な鍵をすでに手にしていることになる。

生物の完全なゲノム配列（ゲノムシーケンス）を決定することは、バイオインフォマティクスの中心的課題である。しかし、我々は依然として「顕微鏡で一塩基ごとにズームして読み取る」ような技術は持っていない。その代わりに、研究者は化学的方法を用いて、より小さなDNA断片（リード）を生成・同定することができる。

大量のリードを同一ゲノムの複数コピーから得たのち、目的とするゲノムをそれらの断片から再構築する。この過程が**フラグメントアセンブリ**である（図1参照）。

---

### 問題設定

一群の文字列が与えられたとき、それらすべてを部分文字列として含むより大きな文字列を**スーパー文字列**という。

最小仮定の原則（パーシモニー）に基づけば、リード集合に対する最短スーパー文字列が候補染色体となる。

**与えられるもの**  
- 高々50本のDNA文字列（長さはほぼ等しく、最大1 kbpまで）。  
- FASTA形式。  
- すべて同一の線状染色体の同一鎖由来のリードと仮定。  

**保証される条件**  
- リードを「長さの半分を超えて重なる部分」でつなぎ合わせることで、全染色体を一意に復元できる。

**求めるもの**  
- 与えられたすべての文字列を含む最短スーパー文字列（すなわち復元染色体配列）。

---

### サンプルデータセット

```
>Rosalind_56
ATTAGACCTG
>Rosalind_57
CCTGCCGGAA
>Rosalind_58
AGACCTGCCG
>Rosalind_59
GCCGGAATAC
```

### サンプル出力

```
ATTAGACCTGCCGGAATAC
```

---

### 補足情報

フラグメントアセンブリの目標は完全なゲノムを得ることであるが、実際には染色体のいくつかの連続領域（コンティグ）を構築するにとどまる。また、すべてのリードが同一の鎖由来であるという仮定は現実的ではなく、実際の研究では各リードがどのDNA鎖から得られたかは不明である。


In [6]:
fasta = """>Rosalind_56
ATTAGACCTG
>Rosalind_57
CCTGCCGGAA
>Rosalind_58
AGACCTGCCG
>Rosalind_59
GCCGGAATAC
"""

In [7]:
sequences = []
for line in fasta.splitlines():
    if not line.startswith(">"):
        sequences.append(line.strip())

print(sequences)

['ATTAGACCTG', 'CCTGCCGGAA', 'AGACCTGCCG', 'GCCGGAATAC']


In [ ]:
from itertools import combinations
from collections import deque

contig = deque()
contig.append(sequences[0])

for _ in range(len(sequences) - 1):
    max_overlap = -1
    max_seq = ""
    for seq1, seq2 in combinations(sequences, 2):
        if seq1 in contig or seq2 in contig:
            overlap = min(len(seq1), len(seq2)) - 1
            while overlap > 0:
                if seq1 in contig and seq1[-overlap:] == seq2[:overlap]:
                    if overlap > max_overlap:
                        max_overlap = overlap
                        max_seq = seq2
                elif seq2 in contig and seq2[-overlap:] == seq1[:overlap]:
                    if overlap > max_overlap:
                        max_overlap = overlap
                        max_seq = seq1
                overlap -= 1
    if contig[0] in sequences and contig[0] == max_seq:
        contig.appendleft(max_seq[max_overlap:])
    else:
        contig.append(max_seq[max_overlap:])
    sequences.remove(max_seq)
print("".join(contig))

ATTAGACCTGCCGCCGGAACCGGAATAC
